# Neural Bag of Words for Amazon Reviews

In [1]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.8"))

# This creates a symbolic link on python in tmp directory

# Helper libraries for 's instance
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io

# Code for this assignment

import models

import nltk
from nltk.tokenize import word_tokenize

## Word Vectors

In [2]:
from pathlib import Path
import os.path
wordsList = np.load(os.path.join(str(Path.home()), '.kaggle/wordvectors/pretrained_glove/wordsList.npy'))
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load(os.path.join(str(Path.home()), '.kaggle/wordvectors/pretrained_glove/wordVectors.npy'))

## Load Reviews

In [3]:
review_df = pd.read_csv('~/.kaggle/datasets/snap/amazon-fine-food-reviews/Reviews.csv', encoding='utf8')
review_df = review_df.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Time', 'Summary'], axis=1)

In [4]:
# Reduce size for development
#numReviews = 100
#review_df = review_df.loc[0:numReviews-1]

Remove reviews where the score is 3.  This is an ambiguous case for binary classification.

In [5]:
review_df = review_df[review_df.Score != 3]
review_df.reset_index(inplace=True)

## Tokenize

In [6]:
review_df['Tokens'] = review_df['Text'].apply(lambda text: word_tokenize(text.lower()))

## Train, Dev, Test data

In [7]:
# Train with 60%, , Dev: 10%, Test: 30%
train_percent = 0.6
dev_percent = 0.1
test_percent = 0.3

# Get indicies of the rows in the dataframe for training and testing
train_lower_index = 0
train_upper_index = train_lower_index + round(len(review_df)*train_percent)
dev_lower_index   = train_upper_index+1
dev_upper_index   = dev_lower_index + round(len(review_df)*dev_percent)
test_lower_index  = dev_upper_index+1
test_upper_index  = len(review_df)-1

In [8]:
maxSeqLength = 267  # Determined by EDA

In [9]:
word_ids = np.zeros([review_df.shape[0], maxSeqLength], dtype=np.int32)
word_ids_ns = np.zeros([review_df.shape[0]], dtype=np.int32)
word_ids_labels = np.zeros([review_df.shape[0]], dtype=np.int32)

In [10]:
for sentence_index, row in review_df.iterrows():
    
    word_index = 0
    
    for word in row['Tokens']:

        try:
            word_ids[sentence_index][word_index] = wordsList.index(word)
        except ValueError:
            word_ids[sentence_index][word_index] = 399999 #Vector for unkown words
        
        word_index = word_index + 1

        if word_index == maxSeqLength:
            break

    word_ids_ns[sentence_index] = word_index

    if row['Score'] > 3:
        word_ids_labels[sentence_index] = 1
    elif row['Score'] < 3:
        word_ids_labels[sentence_index] = 0

In [11]:
def Split(lower_idx, upper_idx):
    return word_ids[lower_idx:upper_idx], word_ids_ns[lower_idx:upper_idx], word_ids_labels[lower_idx:upper_idx]

In [12]:
train_x, train_ns, train_y = Split(train_lower_index, train_upper_index)
dev_x,   dev_ns,   dev_y   = Split(dev_lower_index, dev_upper_index)
test_x,  test_ns,  test_y  = Split(test_lower_index, test_upper_index)

## Model Parameters

In [13]:
import models; reload(models)

<module 'models' from '/home/matt/w266_final_project/NBOW/models.py'>

In [14]:
# Specify model hyperparameters as used by model_fn
model_params = dict(V=len(wordsList),
                    embed_dim=50,
                    hidden_dims=[25],
                    num_classes=2,      # 2 for binary classifier
                    encoder_type='bow',
                    lr=0.1,
                    optimizer='adagrad',
                    beta=0.01,
                    dropout_rate=0.1)  # fill this in

## Training Parameters

In [15]:
# Specify training schedule
train_params = dict(batch_size=32,
                    total_epochs=20,
                    eval_every=2)  # fill this in

In [16]:
checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")

if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
#ds.vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

In [17]:
model = tf.estimator.Estimator(model_fn=models.classifier_model_fn,
                               params=model_params,
                               model_dir=checkpoint_dir)

print("\nTo view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006\n")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_session_config': None, '_log_step_count_steps': 100, '_task_id': 0, '_service': None, '_save_summary_steps': 100, '_model_dir': '/tmp/tf_bow_sst_20180717-1728', '_evaluation_master': '', '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f109e4fb940>, '_save_checkpoints_secs': 600, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_master': '', '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_tf_random_seed': None, '_train_distribute': None, '_num_worker_replicas': 1}

To view training (once it starts), run:

    tensorboard --logdir='/tmp/tf_bow_sst_20180717-1728' --port 6006

Then in your browser, open: http://localhost:6006



In [18]:
train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_ns},
                    y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'],
                    shuffle=True,
                    seed=42)

dev_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": dev_x, "ns": dev_ns},
                    y=dev_y,
                    batch_size=128,
                    num_epochs=1,
                    shuffle=False)

## Train

In [19]:
for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    model.train(input_fn=train_input_fn)
    model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tf_bow_sst_20180717-1728/model.ckpt.
INFO:tensorflow:step = 1, loss = 1.0592134
INFO:tensorflow:global_step/sec: 101.091
INFO:tensorflow:step = 101, loss = 0.7864195 (0.990 sec)
INFO:tensorflow:global_step/sec: 121.674
INFO:tensorflow:step = 201, loss = 0.9445193 (0.823 sec)
INFO:tensorflow:global_step/sec: 112.952
INFO:tensorflow:step = 301, loss = 0.5846317 (0.884 sec)
INFO:tensorflow:global_step/sec: 115.383
INFO:tensorflow:step = 401, loss = 0.5908887 (0.868 sec)
INFO:tensorflow:global_step/sec: 114.112
INFO:tensorflow:step = 501, loss = 0.59412897 (0.875 sec)
INFO:tensorflow:global_step/sec: 113.806
INFO:tensorflow:step = 601, loss = 0.33580065 (0.880 sec)
INFO:tensorflow:global_step/sec: 116.825
I

INFO:tensorflow:global_step/sec: 113.799
INFO:tensorflow:step = 8001, loss = 0.24766192 (0.880 sec)
INFO:tensorflow:global_step/sec: 117.411
INFO:tensorflow:step = 8101, loss = 0.45663005 (0.851 sec)
INFO:tensorflow:global_step/sec: 117.95
INFO:tensorflow:step = 8201, loss = 0.3395608 (0.847 sec)
INFO:tensorflow:global_step/sec: 120.413
INFO:tensorflow:step = 8301, loss = 0.24136609 (0.831 sec)
INFO:tensorflow:global_step/sec: 121.128
INFO:tensorflow:step = 8401, loss = 0.29774275 (0.826 sec)
INFO:tensorflow:global_step/sec: 119.162
INFO:tensorflow:step = 8501, loss = 0.3203095 (0.841 sec)
INFO:tensorflow:global_step/sec: 118.053
INFO:tensorflow:step = 8601, loss = 0.4368946 (0.846 sec)
INFO:tensorflow:global_step/sec: 118.242
INFO:tensorflow:step = 8701, loss = 0.22556734 (0.847 sec)
INFO:tensorflow:global_step/sec: 114.133
INFO:tensorflow:step = 8801, loss = 0.33870816 (0.873 sec)
INFO:tensorflow:global_step/sec: 117.425
INFO:tensorflow:step = 8901, loss = 0.24375242 (0.852 sec)
INFO

INFO:tensorflow:global_step/sec: 118.786
INFO:tensorflow:step = 16201, loss = 0.34974927 (0.842 sec)
INFO:tensorflow:global_step/sec: 119.755
INFO:tensorflow:step = 16301, loss = 0.2175719 (0.836 sec)
INFO:tensorflow:global_step/sec: 116.593
INFO:tensorflow:step = 16401, loss = 0.51471573 (0.858 sec)
INFO:tensorflow:global_step/sec: 120.267
INFO:tensorflow:step = 16501, loss = 0.15434343 (0.836 sec)
INFO:tensorflow:global_step/sec: 117.267
INFO:tensorflow:step = 16601, loss = 0.1998584 (0.847 sec)
INFO:tensorflow:global_step/sec: 119.4
INFO:tensorflow:step = 16701, loss = 0.47875792 (0.837 sec)
INFO:tensorflow:global_step/sec: 119.434
INFO:tensorflow:step = 16801, loss = 0.2791302 (0.838 sec)
INFO:tensorflow:global_step/sec: 116.169
INFO:tensorflow:step = 16901, loss = 0.25055173 (0.860 sec)
INFO:tensorflow:global_step/sec: 116.543
INFO:tensorflow:step = 17001, loss = 0.13407105 (0.859 sec)
INFO:tensorflow:global_step/sec: 118.469
INFO:tensorflow:step = 17101, loss = 0.3642052 (0.844 s

INFO:tensorflow:step = 23219, loss = 0.460176 (0.923 sec)
INFO:tensorflow:global_step/sec: 114.177
INFO:tensorflow:step = 23319, loss = 0.14848271 (0.875 sec)
INFO:tensorflow:global_step/sec: 114.145
INFO:tensorflow:step = 23419, loss = 0.34020025 (0.879 sec)
INFO:tensorflow:global_step/sec: 113.64
INFO:tensorflow:step = 23519, loss = 0.20910297 (0.876 sec)
INFO:tensorflow:global_step/sec: 116.641
INFO:tensorflow:step = 23619, loss = 0.30172288 (0.858 sec)
INFO:tensorflow:global_step/sec: 111.175
INFO:tensorflow:step = 23719, loss = 0.25215933 (0.898 sec)
INFO:tensorflow:global_step/sec: 114.149
INFO:tensorflow:step = 23819, loss = 0.16695473 (0.876 sec)
INFO:tensorflow:global_step/sec: 115.296
INFO:tensorflow:step = 23919, loss = 0.15160641 (0.877 sec)
INFO:tensorflow:global_step/sec: 113.316
INFO:tensorflow:step = 24019, loss = 0.33834547 (0.873 sec)
INFO:tensorflow:global_step/sec: 118.655
INFO:tensorflow:step = 24119, loss = 0.28974828 (0.842 sec)
INFO:tensorflow:global_step/sec: 1

INFO:tensorflow:global_step/sec: 114.791
INFO:tensorflow:step = 31419, loss = 0.3115425 (0.871 sec)
INFO:tensorflow:global_step/sec: 113.542
INFO:tensorflow:step = 31519, loss = 0.20962685 (0.883 sec)
INFO:tensorflow:global_step/sec: 114.852
INFO:tensorflow:step = 31619, loss = 0.3751669 (0.869 sec)
INFO:tensorflow:global_step/sec: 118.469
INFO:tensorflow:step = 31719, loss = 0.25038916 (0.846 sec)
INFO:tensorflow:global_step/sec: 115.021
INFO:tensorflow:step = 31819, loss = 0.22831964 (0.868 sec)
INFO:tensorflow:global_step/sec: 116.985
INFO:tensorflow:step = 31919, loss = 0.12482907 (0.854 sec)
INFO:tensorflow:global_step/sec: 113.525
INFO:tensorflow:step = 32019, loss = 0.36151224 (0.881 sec)
INFO:tensorflow:global_step/sec: 119.497
INFO:tensorflow:step = 32119, loss = 0.2893338 (0.837 sec)
INFO:tensorflow:global_step/sec: 117.35
INFO:tensorflow:step = 32219, loss = 0.2553883 (0.854 sec)
INFO:tensorflow:global_step/sec: 116.741
INFO:tensorflow:step = 32319, loss = 0.24517801 (0.858 

...

INFO:tensorflow:step = 161645, loss = 0.30801946 (0.833 sec)
INFO:tensorflow:global_step/sec: 123.16
INFO:tensorflow:step = 161745, loss = 0.20645265 (0.812 sec)
INFO:tensorflow:global_step/sec: 118.74
INFO:tensorflow:step = 161845, loss = 0.16553363 (0.842 sec)
INFO:tensorflow:global_step/sec: 115.304
INFO:tensorflow:step = 161945, loss = 0.1601947 (0.867 sec)
INFO:tensorflow:global_step/sec: 119.844
INFO:tensorflow:step = 162045, loss = 0.3355581 (0.835 sec)
INFO:tensorflow:global_step/sec: 117.521
INFO:tensorflow:step = 162145, loss = 0.26059365 (0.852 sec)
INFO:tensorflow:global_step/sec: 116.415
INFO:tensorflow:step = 162245, loss = 0.24255134 (0.858 sec)
INFO:tensorflow:global_step/sec: 116.778
INFO:tensorflow:step = 162345, loss = 0.2330941 (0.860 sec)
INFO:tensorflow:global_step/sec: 117.408
INFO:tensorflow:step = 162445, loss = 0.16208252 (0.848 sec)
INFO:tensorflow:global_step/sec: 116.12
INFO:tensorflow:step = 162545, loss = 0.14673892 (0.861 sec)
INFO:tensorflow:global_step

INFO:tensorflow:global_step/sec: 116.677
INFO:tensorflow:step = 169745, loss = 0.24503757 (0.857 sec)
INFO:tensorflow:global_step/sec: 120.088
INFO:tensorflow:step = 169845, loss = 0.18952528 (0.832 sec)
INFO:tensorflow:global_step/sec: 117.639
INFO:tensorflow:step = 169945, loss = 0.1363526 (0.850 sec)
INFO:tensorflow:global_step/sec: 117.965
INFO:tensorflow:step = 170045, loss = 0.31273025 (0.849 sec)
INFO:tensorflow:global_step/sec: 115.627
INFO:tensorflow:step = 170145, loss = 0.28737253 (0.863 sec)
INFO:tensorflow:global_step/sec: 119.613
INFO:tensorflow:step = 170245, loss = 0.23928882 (0.838 sec)
INFO:tensorflow:global_step/sec: 117.478
INFO:tensorflow:step = 170345, loss = 0.21002708 (0.849 sec)
INFO:tensorflow:global_step/sec: 118.419
INFO:tensorflow:step = 170445, loss = 0.17124546 (0.844 sec)
INFO:tensorflow:global_step/sec: 117.974
INFO:tensorflow:step = 170545, loss = 0.16955736 (0.848 sec)
INFO:tensorflow:global_step/sec: 117.875
INFO:tensorflow:step = 170645, loss = 0.28

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180717-1728/model.ckpt-177462
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-18-00:54:51
INFO:tensorflow:Saving dict for global step 177462: accuracy = 0.9391605, cross_entropy_loss = 0.17173782, global_step = 177462, loss = 0.31597835
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180717-1728/model.ckpt-177462
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 177463 into /tmp/tf_bow_sst_20180717-1728/model.ckpt.
INFO:tensorflow:step = 177463, loss = 0.41262054
INFO:tensorflow:global_step/sec: 101.203
INFO:tensorflow:step = 177563, loss = 0.236345 (0.984 sec)
INFO:tensorflow

INFO:tensorflow:global_step/sec: 115.678
INFO:tensorflow:step = 184763, loss = 0.29160124 (0.865 sec)
INFO:tensorflow:global_step/sec: 115.499
INFO:tensorflow:step = 184863, loss = 0.14080857 (0.865 sec)
INFO:tensorflow:global_step/sec: 116.018
INFO:tensorflow:step = 184963, loss = 0.13628009 (0.862 sec)
INFO:tensorflow:global_step/sec: 116.141
INFO:tensorflow:step = 185063, loss = 0.14507467 (0.861 sec)
INFO:tensorflow:global_step/sec: 114.566
INFO:tensorflow:step = 185163, loss = 0.14570624 (0.873 sec)
INFO:tensorflow:global_step/sec: 110.27
INFO:tensorflow:step = 185263, loss = 0.18058497 (0.907 sec)
INFO:tensorflow:global_step/sec: 114.068
INFO:tensorflow:step = 185363, loss = 0.2696957 (0.876 sec)
INFO:tensorflow:global_step/sec: 113.133
INFO:tensorflow:step = 185463, loss = 0.20768288 (0.884 sec)
INFO:tensorflow:global_step/sec: 109.349
INFO:tensorflow:step = 185563, loss = 0.32573012 (0.917 sec)
INFO:tensorflow:global_step/sec: 112.091
INFO:tensorflow:step = 185663, loss = 0.263

INFO:tensorflow:global_step/sec: 113.289
INFO:tensorflow:step = 192863, loss = 0.13772458 (0.883 sec)
INFO:tensorflow:global_step/sec: 109.449
INFO:tensorflow:step = 192963, loss = 0.11144298 (0.914 sec)
INFO:tensorflow:global_step/sec: 109.181
INFO:tensorflow:step = 193063, loss = 0.1829312 (0.916 sec)
INFO:tensorflow:global_step/sec: 112.959
INFO:tensorflow:step = 193163, loss = 0.16049416 (0.885 sec)
INFO:tensorflow:global_step/sec: 113.609
INFO:tensorflow:step = 193263, loss = 0.33183205 (0.881 sec)
INFO:tensorflow:global_step/sec: 113.696
INFO:tensorflow:step = 193363, loss = 0.28532064 (0.878 sec)
INFO:tensorflow:global_step/sec: 111.18
INFO:tensorflow:step = 193463, loss = 0.20312637 (0.900 sec)
INFO:tensorflow:global_step/sec: 115.866
INFO:tensorflow:step = 193563, loss = 0.199377 (0.864 sec)
INFO:tensorflow:global_step/sec: 118.046
INFO:tensorflow:step = 193663, loss = 0.34694678 (0.849 sec)
INFO:tensorflow:global_step/sec: 113.367
INFO:tensorflow:step = 193763, loss = 0.19462

## Test

In [20]:
test_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": test_x, "ns": test_ns},
                    y=test_y,
                    batch_size=128,
                    num_epochs=1,
                    shuffle=False)

eval_metrics = model.evaluate(input_fn=test_input_fn, name="test")
print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-18-00:57:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180717-1728/model.ckpt-197180
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-18-00:58:03
INFO:tensorflow:Saving dict for global step 197180: accuracy = 0.9365166, cross_entropy_loss = 0.18041915, global_step = 197180, loss = 0.32567444
Accuracy on test set: 93.65%
